In [0]:
!pip install tensorflow

import warnings
import pandas as pd
import mlflow
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from mlflow.models.signature import infer_signature

warnings.filterwarnings("ignore")

In [0]:
wind_farm_data = pd.read_csv("https://github.com/dbczumar/model-registry-demo-notebook/raw/master/dataset/windfarm_data.csv", index_col=0)
display(wind_farm_data)

In [0]:
def get_training_data():
  training_data = pd.DataFrame(wind_farm_data["2014-01-01":"2018-01-01"])
  X = training_data.drop(columns="power")
  y = training_data["power"]
  return X, y

In [0]:
import numpy as np

X_train, y_train = get_training_data()

def train_keras_model(X, y):
  import tensorflow.keras
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Dense

  model = Sequential()
  model.add(Dense(100, input_shape=(X_train.shape[-1],), activation="relu", name="hidden_layer"))
  model.add(Dense(100, input_shape=(X_train.shape[-1],), activation="relu", name="hidden_layer2"))
  model.add(Dense(100, input_shape=(X_train.shape[-1],), activation="relu", name="hidden_layer3"))
  model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")

  hist = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_split=.2)
  final_val_loss = float(hist.history["val_loss"][-1])
  mlflow.log_metric("final_val_loss", final_val_loss)
  mlflow.log_metric("final_val_rmse", np.sqrt(final_val_loss))

  return model

In [0]:
with mlflow.start_run(run_name="Windfarm 26_11"):

  mlflow.log_param("epochs", 200)
  mlflow.log_param("batch_size", 64)
  mlflow.log_param("validation_split", .2)
  mlflow.log_param("number_hidden_layers", 3)
  model = train_keras_model(X_train, y_train)

  X_train_np = X_train.to_numpy() if hasattr(X_train, "to_numpy") else X_train
  y_train_np = y_train.to_numpy() if hasattr(y_train, "to_numpy") else y_train
  signature = infer_signature(X_train_np, y_train_np)

In [0]:
sample = wind_farm_data.sample()
display(sample)
X = sample.drop(columns="power")
model.predict(X)

temperature_00,wind_direction_00,wind_speed_00,temperature_08,wind_direction_08,wind_speed_08,temperature_16,wind_direction_16,wind_speed_16,power
15.9102996190389,261.9476,2.5993335,21.365122477213543,259.47485,2.0527923,23.30014451344808,263.67395,5.1948485,698.8562


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


array([[678.5895]], dtype=float32)

In [0]:
model_info = mlflow.tensorflow.log_model(model, "model", signature=signature)

## Load the model

In [0]:
import mlflow
logged_model = 'runs:/d939eb3069b4493598083c4ae90e2bc1/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)


wind_farm_data = pd.read_csv("https://github.com/dbczumar/model-registry-demo-notebook/raw/master/dataset/windfarm_data.csv", index_col=0)
sample = wind_farm_data.sample()
print(sample)
X = sample.drop(columns="power")

prediction = loaded_model.predict(X)
prediction